In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Lab7_RDD_EX1").master("spark://spark-master:7077").getOrCreate() 
sc = spark.sparkContext
sc.setLogLevel('FATAL')
lines_rdd = spark.sparkContext.textFile("/home/jovyan/data/wordcount.txt") # create a RDD from a text file

## Find the most frequent word

In [2]:
from operator import add

def myMapFun1(x):
    return (x, 1)

def myMapFun2(x):
    return (x[1], x[0])

# MapReduce Job 1
word_counts = lines_rdd.flatMap(lambda x: x.split(' ')) \
                  .map(myMapFun1) \
                  .reduceByKey(add)

# MapReduce Job 2
word_counts_ordered = word_counts.map(myMapFun2).sortByKey(ascending=False)
top_word_tuple = word_counts_ordered.take(1)[0]  # also there is a function top
print(type(top_word_tuple))
print("Top word is {}".format(top_word_tuple[1]))

<class 'tuple'>
Top word is the


## Use of Custom Reduce Function

In [3]:
# The function should be associative and commutative. The first parameter is the return value from the previous execution of the reduce function
def custom_add(accum,x):
    return accum+x

In [4]:
word_counts_2 = lines_rdd.flatMap(lambda x: x.split(' ')) \
                  .map(myMapFun1) \
                  .reduceByKey(custom_add)
print(word_counts_2.map(myMapFun2).sortByKey(ascending=False).take(1))

[(22, 'the')]


In [5]:
# Stop the spark context
spark.stop()

## How to calculate average and max/min

https://stackoverflow.com/questions/51236850/how-to-find-an-average-for-a-spark-rdd

https://stackoverflow.com/questions/44176782/spark-python-how-to-use-reduce-by-key-to-get-minmum-maximum-values